In [17]:
# script used to get tweets for interested hashtags
#Read in our list of unique hashtags
#Look for hashtags that match our keywords

In [18]:
topic_keywords = [
'college',\
'marijuana',\
'flintwater',\
'familyleave',\
'clintonemails',\
'benghazi',\
'zika',\
'billclinton',\
'obama',\
'veterans',\
'torture',\
'homelandsecurity',\
'nationalsecurity',\
'socialsecurity',\
'socialism',\
'guncontrol',\
'healthcare',\
'cnn_news',\
'cnn_debate',\
'syria',\
'refugees',\
'iran',\
'iraq',\
'isis',\
'israel',\
'environment',\
'climate'
]

In [19]:
#open the .csv file with the set of all hashtags over the 40 days
import pandas as pd

htfile = pd.read_csv('hashtags_alldates.csv')
htfile.head()


,hashtag,count
0,trump,1864957
1,gopdebate,1681434
2,feelthebern,1432801
3,demdebate,1066694
4,trump2016,1038620


In [20]:
# Write out the hashtags that match the keywords in separate files, one per keyword.

httopic={}

for keyword in topic_keywords:
    
    df = htfile[htfile['hashtag'].str.contains(keyword)==True]
    df.to_csv('keywords/'+keyword+'.csv')

#outfile=topic_keywords[0] +'.csv' 
outfile='keywords/nationalsecurity.csv'
!cat $outfile

,hashtag,count
2838,nationalsecurity,809
21716,nationalsecurityfirst,49
31572,bordersecurityisnationalsecurity,28
44911,tumpisadangertonationalsecurity,16
65238,nationalsecurityfail,9
95074,nationalsecuritycouncil,5
116611,nationalsecurityrisk,4
119611,nationalsecurityissue,4
136529,nationalsecurityagency,3
151719,endangersusnationalsecurity,3
152669,nationalsecuritymatters,3
161274,nationalsecurityadviser,2
168615,nationalsecurityincompetency,2
170565,snationalsecurityadvisor,2
176395,nationalsecuritythreat,2
205247,nationalsecurityadvisor,2
206609,nationalsecurityplan,2
265341,nationalsecurityfb,1
282326,trump4nationalsecurity,1
299212,matterofnationalsecurity,1
324456,threat2nationalsecurity,1
336787,greatestnationalsecuritythreat,1
340397,nationalsecurityadvisorycouncil,1
354275,gopnationalsecurityleaders,1
369604,harmednationalsecurity,1
381277,walltradenationalsecurityjobseconomyleader,1
396494,nationalsecurityleaders,1
404009,nationalsecurity101,1
433475,wouldyoutrusthernational

In [8]:
# Create a set of selected hashtags (manually selected from files)

hts={}

hts['college']="""['freecollege','collegeforall','college4all','freecommunitycollege','tuitionfreecollege']"""
hts['marijuana']="""['marijuana','medicalmarijuana','legalizemarijuana','medicalmarijuanamovement']"""
hts['flintwater']="""['flintwater','flintwaterpoison','flintwaterworks']"""
hts['familyleave']=['paidfamilyleave','paidleave','parentalleave','voteleave','paidmaternityleave','familyleave']
hts['environment']=['environment','environmentalracism','environmentaljustice','climatechange','climate','actonclimate']
hts['clintonemails']=['clintonemails','hillaryclintonemails','clintonemailscandal']
hts['benghazi']=['benghazi','benghazimatters','benghazicommittee','benghazi4','rememberbenghazi',\
                 'hillarybenghaziclinton']
hts['zika']=['zika','zikavirus']
hts['billclinton']=['billclinton','movebillgetouttheway','arrestbillclinton','billary','billclintonrapes']
hts['obama']=['obama','thanksobama','barackobama','presidentobama','republicanobama']
hts['veterans']=['veterans','veteransfortrump','veterans4trump','veteransagainsttrump',\
                 'veteransforbernie']
hts['torture']=['torture','supporttorture','realtorture','tortureisawarcrime','stoptorture']
hts['homelandsecurity']=['bordersecurity','homelandsecurity','bordersecurityisnationalsecurity']
hts['socialsecurity']=['socialsecurity','savesocialsecurity','expandsocialsecurity']
hts['nationalsecurity']=['nationalsecurity','nationalsecurityfirst','nationalsecurityfail']
hts['socialism']=['socialism','socialismchecklist','democraticsocialism','socialismsucks','trysocialism','nosocialism']
hts['guncontrol']=['guncontrol','guncontrolnow','guncontrolkills']
hts['healthcare']=['obamacare','universalhealthcare','healthcareforall','healthcare4all','singlepayerhealthcare']
hts['cnn_news']=['cnn','cnn_news','cnnnews', 'cnnelection','cnnheadlines','cnnlive']
hts['cnn_debate']=['cnndebate','cnntownhall','cnngopdebate']
hts['syria']=['syria','syrian','syrians','dontbombsyria','syriac']
hts['refugees']=['syrianrefugees','nosyrianrefugees']
hts['iran']=['iran','irandeal','iranelections','stopexecutioniran','irantalks','noirandeal','nonucleariran']
hts['iraq']=['iraq','iraqwar','iraqi']
hts['isis']=['isis','noisis','stopisis']
hts['israel']=['israel','istandwithisrael','theisraelcompany','standwithisrael','israeltheregion','israeli','israelnews']
hts['poverty'] = ['poverty']
hts['conservatism'] = ['conservatism']
hts['corruption'] = ['corruption']
hts['supremecourt'] = ['supremecourt']
hts['ronaldreagan'] = ['ronaldreagan']

In [9]:
# now get tweets that contain the hashtags.

import csv
import pandas as pd
from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://sauce5.sdsc.edu",auth=basic_auth("neo4j", "lajolla"))

query="""MATCH (t:Tweet_{0})<-[:TAGS]-(h:Hashtag_{0}) \
WHERE h.name in {1} and not exists(t.retweet_id) RETURN h.name, t.id_str, t.text LIMIT 200"""

partitions=['2016_2_9','2016_2_23','2016_3_7']

df2 = pd.DataFrame()

with open('hashtagtweets_g4.csv', 'wb') as csvfile:
    tweetwriter = csv.writer(csvfile)
    
    for partition in partitions:
        
        #for keyword in hts.keys():
        for keyword in ['poverty','corruption','ronaldreagan','conservatism','supremecourt']:

            query_formatted = query.format(partition,hts[keyword])
            print query_formatted

            session = driver.session()
            result = session.run(query_formatted)


            for record in result:
                print record
                tweetwriter.writerow([keyword,record[1].encode("utf-8"),record[2].encode("utf-8")])

            session.close()

MATCH (t:Tweet_2016_2_9)<-[:TAGS]-(h:Hashtag_2016_2_9) WHERE h.name in ['poverty'] and not exists(t.retweet_id) RETURN h.name, t.id_str, t.text LIMIT 200
<Record h.name=u'poverty' t.id_str=u'697162850834321408' t.text=u'"#obama et al &amp; #liberal #democrats the real enemy of #America. Casual analysis of 50yrs failed &amp; perpetuated #poverty &amp; #racism self-evident "'>
<Record h.name=u'poverty' t.id_str=u'697162732701929473' t.text=u'"@BernieSanders has passed bills both big &amp; small that reshaped American policy on key issues like #poverty, the #environment &amp; #health care. "'>
<Record h.name=u'poverty' t.id_str=u'697158791456894976' t.text=u'"Hillary Clinton\u2019s War on the #Poor https://t.co/LQ1YHycBoH #HillaryClinton #poverty #Haiti #guncontrol #workingpoor #minimumwage #Bernie2016 "'>
<Record h.name=u'poverty' t.id_str=u'697157352990986240' t.text=u'"Top Economist Says Bernie Sanders\u2019 Plan Will Raise Median Income $22,000 and Create 26 Million Jobs https://t.co/